<a href="https://colab.research.google.com/github/Nathruth/credit-card-fraud-detection/blob/master/NNcreditcard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch onnx onnxruntime scikit-learn pandas numpy matplotlib


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd

df = pd.read_csv("creditcard.csv")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
import torch
import torch.nn as nn

class FraudMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout=0.0):
        super().__init__()
        layers = []

        prev_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = h

        layers.append(nn.Linear(prev_dim, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


In [ ]:
from tqdm.notebook import trange
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import pandas as pd

def train_model(model, lr, epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCEWithLogitsLoss()

    dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train.values, dtype=torch.float32)
    )
    loader = DataLoader(dataset, batch_size=256, shuffle=True)

    model.train()
    epoch_losses = []

    for epoch in trange(epochs, desc="Training Epochs", leave=True):
        epoch_loss = 0
        for xb, yb in loader:
            optimizer.zero_grad()
            preds = model(xb).squeeze()
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * xb.size(0)

        avg_loss = epoch_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    # Display after training
    df_losses = pd.DataFrame({"epoch": range(1, epochs+1), "loss": epoch_losses})
    display(df_losses)
    return model, df_losses



In [ ]:
from sklearn.metrics import roc_auc_score

def evaluate(model):
    model.eval()
    with torch.no_grad():
        logits = model(torch.tensor(X_test, dtype=torch.float32)).squeeze()
        probs = torch.sigmoid(logits).numpy()
    return roc_auc_score(y_test, probs)


In [ ]:
model = FraudMLP(input_dim=X_train.shape[1], hidden_dims=[32], dropout=0.0)
model, losses = train_model(model, lr=1e-3)
score = evaluate(model)




In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses["epoch"], losses["loss"], marker='o')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss per Epoch")
plt.show()


In [ ]:
results = []
results.append({
    "layers": [64, 32],
    "dropout": 0.3,
    "lr": 1e-3,
    "roc_auc": score
})


In [ ]:
from sklearn.metrics import roc_auc_score
import torch

model.eval()  # very important!

with torch.no_grad():
    logits = model(torch.tensor(X_test, dtype=torch.float32)).squeeze()
    probs = torch.sigmoid(logits).numpy()

roc_auc = roc_auc_score(y_test, probs)
print("Neural Network ROC-AUC:", roc_auc)


In [ ]:
results.append({
    "layers": [64,32],
    "dropout": 0.3,
    "lr": 1e-3,
    "roc_auc": roc_auc
})


In [ ]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("roc_auc", ascending=False)
df_results


In [ ]:
# Save CSV
df_results.to_csv("nn_experiments.csv", index=False)

# Download locally from Colab
from google.colab import files
files.download("nn_experiments.csv")


In [ ]:
!pip install onnx onnxruntime onnxscript


In [ ]:
import torch

dummy_input = torch.randn(1, X_train.shape[1])
torch.onnx.export(
    model,
    dummy_input,
    "fraud_mlp.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)
from google.colab import files
files.download("fraud_mlp.onnx")  # download for GitHub
